In [14]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import gc, math
import pickle

from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import Model, load_model, model_from_json

from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

from cnnmodels import vgg_std16_model, preprocess_input, create_rect5, load_img, train_generator, test_generator
from cnnmodels import identity_block, testcv_generator, conv_block, resnet50_model

In [72]:
# Params
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
ROWS, COLS = 224, 224
CHECKPOINT_DIR = 'log/checkpoint02/'
BATCHSIZE = 32
CERV_CLASSES = ['Type_1', 'Type_2', 'Type_3']
nb_perClass = int(BATCHSIZE / len(CERV_CLASSES))
TRAIN_DIR = '../data/train'
TEST_DIR = '../data/test'
DATA_DIR = '../data'
num_class = len(CERV_CLASSES)
full = False
bags = 5

In [3]:
train_datagen = ImageDataGenerator(
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)

In [ ]:
img_ls = []
y_ls = []
for typ in CERV_CLASSES:
    for img in os.listdir(os.path.join(TRAIN_DIR, typ)):
        if img != '.DS_Store':
            img_ls.append(os.path.join(TRAIN_DIR, typ, img))
            y_ls.append(typ)
for typ in CERV_CLASSES:
    for img in os.listdir(os.path.join(DATA_DIR, typ)):
        if img != '.DS_Store':
            img_ls.append(os.path.join(DATA_DIR, typ, img))
            y_ls.append(typ)
train_all  = pd.DataFrame({'class': y_ls, 'img': img_ls, })[['img', 'class']]

In [60]:
img_ls = []
for img in os.listdir(TEST_DIR):
    if img != '.DS_Store':
        img_ls.append(os.path.join(TEST_DIR, img))
test_df  = pd.DataFrame({'img': img_ls}) 

In [5]:
train_all.tail(3)

,img,class
8209,../data/Type_3/5391.jpg,Type_3
8210,../data/Type_3/4116.jpg,Type_3
8211,../data/Type_3/568.jpg,Type_3


In [43]:
def train_generator(datagen, df):
    while 1:
        batch_x = np.zeros((BATCHSIZE, ROWS, COLS, 3), dtype=K.floatx())
        batch_y = np.zeros((BATCHSIZE, len(CERV_CLASSES)), dtype=K.floatx())
        fn = lambda obj: obj.loc[np.random.choice(obj.index, size=nb_perClass, replace=False),:]
        batch_df = df.groupby('class', as_index=True).apply(fn)
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = row[0]
            typ_class = row[1]
            img = Image.open(image_file).resize((ROWS, COLS))
            img = img.convert('RGB')
            x = np.asarray(img, dtype=K.floatx())
            #x = datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            batch_y[i,CERV_CLASSES.index(typ_class)] = 1
            i += 1
        #return (batch_x, batch_y)
        yield (batch_x.transpose(0, 3, 1, 2), batch_y)

In [7]:
# Split into train and valid
valid_set = pd.read_csv("../val_images.csv", header = None, names = ['img']).img.tolist()
valid_set[-4:]

['Type_2/3498.jpg', 'Type_2/1341.jpg', 'Type_3/6017.jpg', 'Type_2/5629.jpg']

In [8]:
valid_df = train_all[train_all['img'].str.replace('../data/', '').isin(valid_set)]
if full == True:
    train_df = train_all
else:
    train_df = train_all[~train_all['img'].isin(valid_set)]
samples_per_epoch=BATCHSIZE*math.ceil(train_df.groupby('class').size()['Type_2']/nb_perClass)
print(train_df.shape)
print(valid_df.shape)

(8212, 2)
(1642, 2)


In [9]:
samples_per_epoch = 5000

In [10]:
# Make our validation set
l = valid_df.groupby('class').size()
valid_x = np.zeros((valid_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
valid_y = np.zeros((valid_df.shape[0], len(CERV_CLASSES)), dtype=K.floatx())
i = 0
for index,row in valid_df.iterrows():
    row = row.tolist()
    image_file = row[0]
    typ_class = row[1]
    img = Image.open(image_file).resize((ROWS, COLS))
    img = img.convert('RGB')
    x = np.asarray(img, dtype=K.floatx())
    # x = datagen.random_transform(x)
    x = preprocess_input(x)
    valid_x[i] = x
    valid_y[i,CERV_CLASSES.index(typ_class)] = 1
    i += 1
valid_x = valid_x.transpose(0, 3, 1, 2)

In [49]:
def test_generator(df, datagen, batch_size = BATCHSIZE):
    n = df.shape[0]
    batch_index = 0
    while 1:
        current_index = batch_index * batch_size
        if n >= current_index + batch_size:
            current_batch_size = batch_size
            batch_index += 1    
        else:
            current_batch_size = n - current_index
            batch_index = 0        
        batch_df = df[current_index:current_index+current_batch_size]
        batch_x = np.zeros((batch_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = row[0]
            # typ_class = row[1]
            img = Image.open(image_file).resize((ROWS, COLS))
            img = img.convert('RGB')
            x = np.asarray(img, dtype=K.floatx())
            # x = datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            i += 1
        if batch_index%100 == 0: print(batch_index)
        # return (batch_x.transpose(0, 3, 1, 2))
        yield(batch_x.transpose(0, 3, 1, 2))

In [11]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')        
model_checkpoint = ModelCheckpoint(filepath=CHECKPOINT_DIR+'weights.{epoch:03d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
# learningrate_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', epsilon=0.001, cooldown=0, min_lr=0)
# tensorboard = TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=False, write_images=True)

In [12]:
print "Model creation... "
nb_epoch = 2
model = resnet50_model(ROWS, COLS, channel, num_class)
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[-3:]:
    layer.trainable = True

# Start Fine-tuning
print "Fine tune part 1"
model.fit_generator(train_generator(train_datagen, train_df),
          nb_epoch=nb_epoch,
          samples_per_epoch=samples_per_epoch, #50000,
          verbose=1,
          validation_data=(valid_x, valid_y),
          callbacks=[early_stopping, model_checkpoint],
          )

Model creation... 
Fine tune part 1
Epoch 1/2
4992/5000 [============================>.] - ETA: 1s - loss: 1.0207 - acc: 0.4026

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_loss improved from inf to 1.01069, saving model to log/checkpoint02/weights.000-1.0107.hdf5
5024/5000 [==============================] - 1062s - loss: 1.0188 - acc: 0.4047 - val_loss: 1.0107 - val_acc: 0.5024
Epoch 2/2
5024/5000 [==============================] - 1016s - loss: 0.8877 - acc: 0.5086 - val_loss: 0.9186 - val_acc: 0.5463


In [13]:
# Start Fine-tuning
nb_epoch = 4
print "Fine tune part 1A"
model.fit_generator(train_generator(train_datagen, train_df),
          nb_epoch=nb_epoch,
          samples_per_epoch=samples_per_epoch, #50000,
          verbose=1,
          validation_data=(valid_x, valid_y),
          callbacks=[early_stopping, model_checkpoint],
          )

Fine tune part 1A
Epoch 1/4
5024/5000 [==============================] - 1038s - loss: 0.7995 - acc: 0.5673 - val_loss: 0.9020 - val_acc: 0.5694
Epoch 2/4
5024/5000 [==============================] - 1005s - loss: 0.7590 - acc: 0.6079 - val_loss: 0.7860 - val_acc: 0.6358
Epoch 3/4
5024/5000 [==============================] - 1003s - loss: 0.7079 - acc: 0.6324 - val_loss: 0.7511 - val_acc: 0.6620
Epoch 4/4
5024/5000 [==============================] - 1000s - loss: 0.6513 - acc: 0.6682 - val_loss: 0.6751 - val_acc: 0.7065


In [14]:
#from keras.models import load_model, model_from_json
#model = load_model('log/checkpoint01weights.003-0.7728.hdf5')

In [15]:
#learningrate_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', epsilon=0.001, cooldown=0, min_lr=0)

In [16]:
for layer in model.layers[38:]:
    layer.trainable = True
#model.optimizer.lr = 1e-5
nb_epoch = 10
print "Fine tune part 2"
model.fit_generator(train_generator(train_datagen, df=train_df),
          nb_epoch=nb_epoch,
          samples_per_epoch=samples_per_epoch,
          verbose=1,
          validation_data=(valid_x, valid_y),
          callbacks=[model_checkpoint, early_stopping], # , 
          )

Fine tune part 2
Epoch 1/10
5024/5000 [==============================] - 1017s - loss: 0.5980 - acc: 0.7042 - val_loss: 0.6307 - val_acc: 0.7345
Epoch 2/10
5024/5000 [==============================] - 990s - loss: 0.5568 - acc: 0.7251 - val_loss: 0.5777 - val_acc: 0.7619
Epoch 3/10
5024/5000 [==============================] - 990s - loss: 0.5337 - acc: 0.7406 - val_loss: 0.5398 - val_acc: 0.7917
Epoch 4/10
5024/5000 [==============================] - 991s - loss: 0.4854 - acc: 0.7729 - val_loss: 0.4752 - val_acc: 0.8252
Epoch 5/10
5024/5000 [==============================] - 998s - loss: 0.4431 - acc: 0.7898 - val_loss: 0.4545 - val_acc: 0.8423
Epoch 6/10
5024/5000 [==============================] - 992s - loss: 0.4052 - acc: 0.8055 - val_loss: 0.4113 - val_acc: 0.8575
Epoch 7/10
5024/5000 [==============================] - 987s - loss: 0.3805 - acc: 0.8133 - val_loss: 0.3832 - val_acc: 0.8819
Epoch 8/10
5024/5000 [==============================] - 985s - loss: 0.3414 - acc: 0.8402 - v

In [17]:
nb_epoch = 10
model.fit_generator(train_generator(train_datagen, df=train_df),
          nb_epoch=nb_epoch,
          samples_per_epoch=samples_per_epoch,
          verbose=1,
          callbacks=[early_stopping, model_checkpoint],
          validation_data=(valid_x, valid_y),
          )

Epoch 1/10
5024/5000 [==============================] - 1008s - loss: 0.2658 - acc: 0.8720 - val_loss: 0.2555 - val_acc: 0.9294
Epoch 2/10
5024/5000 [==============================] - 984s - loss: 0.2279 - acc: 0.8897 - val_loss: 0.2285 - val_acc: 0.9452
Epoch 3/10
5024/5000 [==============================] - 985s - loss: 0.2124 - acc: 0.8923 - val_loss: 0.2046 - val_acc: 0.9513
Epoch 4/10
5024/5000 [==============================] - 988s - loss: 0.1895 - acc: 0.8991 - val_loss: 0.1840 - val_acc: 0.9629
Epoch 5/10
5024/5000 [==============================] - 989s - loss: 0.1731 - acc: 0.9082 - val_loss: 0.1595 - val_acc: 0.9708
Epoch 6/10
5024/5000 [==============================] - 982s - loss: 0.1566 - acc: 0.9094 - val_loss: 0.1703 - val_acc: 0.9592
Epoch 7/10
5024/5000 [==============================] - 993s - loss: 0.1436 - acc: 0.9116 - val_loss: 0.1205 - val_acc: 0.9817
Epoch 8/10
5024/5000 [==============================] - 993s - loss: 0.1316 - acc: 0.9164 - val_loss: 0.1334 -

In [20]:
import glob
# files = glob.glob(CHECKPOINT_DIR+'*')
# val_losses = [float(f.split('-')[-1][:-5]) for f in files]
# # weights.009-0.2856.hdf5
# index = val_losses.index(min(val_losses))
# print('Loading model from checkpoints file ' + files[index])
# model = load_model(files[index])

In [26]:
# Hack to solve issue on model loading : https://github.com/fchollet/keras/issues/4044
import glob
import h5py
model_files = sorted(glob.glob('log/checkpoint02/*.hdf5'))
for model_file in model_files:
    print("Update '{}'".format(model_file))
    with h5py.File(model_file, 'a') as f:
        if 'optimizer_weights' in f.keys():
            del f['optimizer_weights']

Update 'log/checkpoint02/weights.000-0.2555.hdf5'
Update 'log/checkpoint02/weights.000-0.6307.hdf5'
Update 'log/checkpoint02/weights.000-0.9020.hdf5'
Update 'log/checkpoint02/weights.000-1.0107.hdf5'
Update 'log/checkpoint02/weights.001-0.2285.hdf5'
Update 'log/checkpoint02/weights.001-0.5777.hdf5'
Update 'log/checkpoint02/weights.001-0.7860.hdf5'
Update 'log/checkpoint02/weights.001-0.9186.hdf5'
Update 'log/checkpoint02/weights.002-0.2046.hdf5'
Update 'log/checkpoint02/weights.002-0.5398.hdf5'
Update 'log/checkpoint02/weights.002-0.7511.hdf5'
Update 'log/checkpoint02/weights.003-0.1840.hdf5'
Update 'log/checkpoint02/weights.003-0.4752.hdf5'
Update 'log/checkpoint02/weights.003-0.6751.hdf5'
Update 'log/checkpoint02/weights.004-0.1595.hdf5'
Update 'log/checkpoint02/weights.004-0.4545.hdf5'
Update 'log/checkpoint02/weights.005-0.4113.hdf5'
Update 'log/checkpoint02/weights.006-0.1205.hdf5'
Update 'log/checkpoint02/weights.006-0.3832.hdf5'
Update 'log/checkpoint02/weights.007-0.3332.hdf5'


### Try a prediction from a single epoch

In [36]:
model = load_model('log/checkpoint02/weights.009-0.2856.hdf5')

In [50]:
test_preds = model.predict_generator(test_generator(test_df, train_datagen), 
                                         val_samples = test_df.shape[0])

0


In [65]:
test_sub = pd.DataFrame(test_preds, columns=CERV_CLASSES)
test_sub['image_name'] = test_df['img'].str.split('/').apply(lambda x: x[-1])
test_sub = test_sub[['image_name'] + CERV_CLASSES ]
test_sub.head(3)

,image_name,Type_1,Type_2,Type_3
0,400.jpg,0.202256,0.129008,0.668735
1,430.jpg,0.088869,0.504176,0.406955
2,303.jpg,0.869445,0.124915,0.005641


In [68]:
import time
timestr = time.strftime("%Y%m%d")
if full:
    subm_name = '../sub/sub_dara_full_resnet_raw_' + timestr + '.csv' #'.csv.gz'
else:
    subm_name = '../sub/sub_dara_part_resnet_raw_' + timestr + '.csv' #'.csv.gz'

#### ... this got LB 0.77

In [71]:
from IPython.lib.display import FileLink
test_sub.to_csv(subm_name, index=False)#, compression='gzip')
FileLink(subm_name)

/home/ubuntu/cervical/sub/sub_dara_part_resnet_raw_20170510.csv

### Bag the predictions from a few epochs

In [93]:
import glob
files = glob.glob(CHECKPOINT_DIR+'*')
# there is apparently overfitting on the later epochs so exclude the epochs where we unfroze the top layers
files = [f for f in files if float(f.split('-')[-1][:-5])>0.26]
val_losses = [float(f.split('-')[-1][:-5]) for f in files]
min_id = np.array(val_losses).argsort()[:bags].tolist()

In [94]:
# Loop the lowest val losses and get a prediction for each
test_preds_ls = []
for index in min_id:
    print('Loading model from checkpoints file ' + files[index])
    test_model = load_model(files[index])
    test_model_name = files[index].split('/')[-2][-1:]+'_'+files[index].split('/')[-1]
    test_preds_ls.append(test_model.predict_generator(test_generator(test_df, train_datagen), 
                                         val_samples = test_df.shape[0])) 
    del test_model
    gc.collect()

Loading model from checkpoints file log/checkpoint02/weights.009-0.2856.hdf5
0
Loading model from checkpoints file log/checkpoint02/weights.008-0.3054.hdf5
0
Loading model from checkpoints file log/checkpoint02/weights.007-0.3332.hdf5
0
Loading model from checkpoints file log/checkpoint02/weights.006-0.3832.hdf5
0
Loading model from checkpoints file log/checkpoint02/weights.005-0.4113.hdf5
0


In [95]:
test_preds = sum(test_preds_ls)/len(test_preds_ls)

In [96]:
test_sub = pd.DataFrame(test_preds, columns=CERV_CLASSES)
test_sub['image_name'] = test_df['img'].str.split('/').apply(lambda x: x[-1])
test_sub = test_sub[['image_name'] + CERV_CLASSES ]
test_sub.head(3)

,image_name,Type_1,Type_2,Type_3
0,400.jpg,0.254422,0.156392,0.589186
1,430.jpg,0.087055,0.594300,0.318645
2,303.jpg,0.845521,0.146083,0.008396


#### .. @LB : 0.76

In [97]:
if full:
    subm_name = '../sub/sub_dara_full_resnet_raw_5xbag_' + timestr + '.csv' #'.csv.gz'
else:
    subm_name = '../sub/sub_dara_part_resnet_raw_5xbag_' + timestr + '.csv' #'.csv.gz'
    
test_sub.to_csv(subm_name, index=False)#, compression='gzip')
FileLink(subm_name)

/home/ubuntu/cervical/sub/sub_dara_part_resnet_raw_5xbag_20170510.csv